In [1]:
import torch
from torchvision import transforms, datasets


In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[.229, 0.224, 0.225])
    ])
}


In [3]:
image_datasets = {
    'train': datasets.ImageFolder("../input/train", data_transforms['train']),
    'val': datasets.ImageFolder("../input/val", data_transforms['val'])
}


In [4]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 188
     Root location: ../input/train
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=PIL.Image.BILINEAR)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ),
 'val': Dataset ImageFolder
     Number of datapoints: 62
     Root location: ../input/val
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=PIL.Image.BILINEAR)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            )}

In [5]:
image_dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=4, shuffle=True, num_workers=0, drop_last=True),
    'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=4, shuffle=False, num_workers=0, drop_last=True),
    }


In [6]:
image_dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7fc0f215e690>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x7fc0f215e610>}

In [7]:
image_datasets['train']

Dataset ImageFolder
    Number of datapoints: 188
    Root location: ../input/train
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=PIL.Image.BILINEAR)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [9]:
from torchvision import datasets, models, transforms

model_ft = models.resnet18(pretrained=False)


In [11]:
import torch.nn as nn
import torch.optim as optim


device = 'cpu'
TARGET_NUM = 10

def get_model(target_num, isPretrained=False):

    model_ft = models.resnet18(pretrained=isPretrained)
    model_ft.fc = nn.Linear(512, target_num)
    model_ft = model_ft.to(device)

model = get_model(TARGET_NUM, isPretrained=False)


In [15]:
model

In [12]:
optimizer = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

criterion = nn.CrossEntropyLoss()


In [13]:
def train_model(model, criterion, optimizer, num_epochs=5, is_saved=False):
    best_acc = 0.0

    for epoch in range(num_epochs):

        for phase in ['train', 'val']:
            print("{}: phase".format(phase))

            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for i,(inputs, labels) in enumerate(image_dataloders[phase]):
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)

                optimizer.zero_grad()

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)

                loss = criterion(outputs, labels)
                print('   loaders:{} kaime'.format(i+1),' loss: {}'.format(loss))

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds==labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print("{} Loss: {:.4f} ACC: {:.4f}".format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                if(is_saved):
                    torch.save(model.state_dict(), './original_model_{}.pth'.format(epoch))
    print("Best val Acc: {:.4f}".format(best_acc))


In [14]:
train_model(model, criterion, optimizer)

train: phase


AttributeError: 'NoneType' object has no attribute 'train'